# Data Sciencie Challengue

## Importe librerias

In [1]:
import requests
import pandas as pd
import numpy as np
import json
import time
import csv
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import datetime
import os

## Funciones

In [57]:

def get_all_category(pais):
    cats = requests.get('https://api.mercadolibre.com/sites/'+pais+'/categories')
    json_data = cats.json()
    df_cats = pd.json_normalize(json_data)
    return df_cats


def get_products_by_category(pais,category, n_fetch=1000):
    base_url = 'https://api.mercadolibre.com/sites/'+pais+'/search'
#    offset = 0
    products = []
    for offset in pam_offset:
        print(offset)
        response = requests.get(base_url, params={'category': category, 'offset': offset})
        data = response.json()
        if not data['results']:
            break
        products.extend(data['results'])

    
    return products


def get_products_detail(product_ids):
    sep = ','
    product_ids = sep.join(product_ids)
    base_url = f'https://api.mercadolibre.com/items?ids={product_ids}'
    response = requests.get(base_url).json()
    item_list = []
    for item in response:
        try:
            item_list.append(item['body'])
        except:
            continue
    return item_list

## Parametros

In [52]:
os.chdir("C:/Users/svescovij/OneDrive - Deloitte (O365D)/Desktop/Proyectos/Manager/Meli/Data")
## Pais para analizar items 
pais='MLC'        
offset_=50           
batch_size = 20
# Atributos rama Json
pam_variable    = ["seller"]


## Carga de categorias

In [16]:
cats = requests.get('https://api.mercadolibre.com/sites/'+pais+'/categories')
json_data=cats.json()
df_cats = pd.json_normalize(json_data)
pam_categoria   = df_cats['id']

## Construccion de tablon Item-seller

In [26]:
# Declaro Dataframe para cosntruir Tablon
df_item = pd.DataFrame()
df_parametro= pd.DataFrame()

pam_offset = []
##
for i in range(0, offset_, 50):
    pam_offset.append(i)
pam_offset.append(offset_)
##
start_time = time.time()
df_item=[]
for cat_id in pam_categoria:
    print(cat_id)
    for offset in pam_offset:
            # Busca en la api por offset y categoria
            url = f'https://api.mercadolibre.com/sites/'+pais+'/search?category={cat_id}&offset={offset}'         
            request = requests.get(url)
            items = request.json()
            # Convierte json en Dataframe
            df = pd.DataFrame(items["results"])
            df['cat_id']=cat_id

           
            for parametro in pam_variable:

                # Crea un nuevo DataFrame con los datos correspondientes al parametro  
                df_variable = pd.json_normalize(df[parametro])
                df_variable = df_variable.add_prefix(parametro+'_')
                
                # Concatena el nuevo DataFrame al DataFrame existente
                df = pd.concat([df, df_variable], axis=1)

            # Uno base con todos los atributos
            if len(df_item) > 0:
                df_item= pd.concat([df_item, df], ignore_index = True)
            else:
                df_item = df
end_time = time.time()

total_time = end_time - start_time

print("El ciclo tardó", total_time/60, "minutos en completarse.")


MLC1747
MLC1512
MLC1403
MLC1071
MLC1367
MLC1368
MLC1743
MLC1384
MLC1246
MLC1039
MLC1051
MLC1648
MLC1144
MLC1500
MLC1276
MLC5726
MLC1000
MLC110931
MLC178483
MLC1574
MLC1499
MLC1459
MLC1182
MLC1132
MLC3025
MLC1168
MLC3937
MLC409431
MLC1540
MLC435280
MLC1430
MLC1953
El ciclo tardó 2.821376613775889 minutos en completarse.


## Construccion de tablon Item-Detalle

In [59]:
start_time = time.time()
product_details = []
ids_list = []
## Obtengo categorias del pais seleccionado
categories=get_all_category(pais)


## Listas de todos los productos por categoria segun parametro de offset y pais
for cat_id in categories['id']:
    print(cat_id)
    products_list = get_products_by_category(pais,cat_id, offset)
    ids_list.extend([product['id'] for product in products_list])

## obtengo detalle de todos los producto de la lista anterior
for step, _ in enumerate(range(0, len(ids_list), batch_size)):
    print(step)
    ids_to_query = ids_list[step*batch_size:batch_size*(step+1)]
    product_details.extend(get_products_detail(ids_to_query))

## Convierto en dataframe lista anterior
data = pd.DataFrame(product_details)

end_time = time.time()

total_time = end_time - start_time

print("El ciclo tardó", total_time/60, "minutos en completarse.")


MLC1747
0
50
MLC1512
0
50
MLC1403
0
50
MLC1071
0
50
MLC1367
0
50
MLC1368
0
50
MLC1743
0
50
MLC1384
0
50
MLC1246
0
50
MLC1039
0
50
MLC1051
0
50
MLC1648
0
50
MLC1144
0
50
MLC1500
0
50
MLC1276
0
50
MLC5726
0
50
MLC1000
0
50
MLC110931
0
50
MLC178483
0
50
MLC1574
0
50
MLC1499
0
50
MLC1459
0
50
MLC1182
0
50
MLC1132
0
50
MLC3025
0
50
MLC1168
0
50
MLC3937
0
50
MLC409431
0
50
MLC1540
0
50
MLC435280
0
50
MLC1430
0
50
MLC1953
0
50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
El ciclo tardó 3.3859435160954794 minutos en c

id                         3178
title                      3178
condition                  3078
thumbnail_id               3178
catalog_product_id         1786
                           ... 
location                    300
seller_contact              300
seller_home_image_url        98
reservation_price           100
reservation_currency_id     100
Length: 92, dtype: int64